In [1]:
import pandas as pd
from datetime import datetime


In [3]:
def get_df_old(file:str, path:str='') -> pd.DataFrame: 
    """
    produces a formatted dataframe with datetime index
    works from 04/10/2018-08/16/2021 - after that the format changes
    """
    if path: 
        file = path + file
    raw = pd.read_csv(file, sep='delimiter', engine='python')

    def rows_as_key_value(row, delete_first=False): 
        """
        returns: [key_name, [values]] for this particular row. 
        necessary for the last row of the raw csv due to an error in the formatting of the csv
        """
        split = lambda string: string.split(',')
        series = list(map(split, row))[0]
        if delete_first: 
            series = series[1:]
        key = series.pop(0)
        return [key, series]

    data = {}
    columns = [raw.columns, raw.iloc[0], raw.iloc[1], raw.iloc[2]]
    for i in range(len(columns)): 
        key, value = rows_as_key_value(columns[i], delete_first=(i==3))
        data.update({key: value})
    
    
    for i in data.keys(): 
        print(len(data[i]))
        
        
    df = pd.DataFrame(dict([(k, pd.Series(v)) for k,v in data.items() ])).iloc[:-1, :] # skip last row - 00:00 of next day
    
    
    
    
    return df
   
def set_dt_index_old(df:pd.DataFrame, date:str)->pd.DataFrame:
    index = list(map(lambda x: datetime.strptime(date + ' ' + x, "%m/%d/%Y %H:%M"), list(df.iloc[:, 0])))
    df.index=pd.DatetimeIndex(index)
    df.drop("Net Demand " + date, axis=1, inplace=True)
    df.index.names = ['Timestamp']
    return df
    
df = get_df_old('CAISO-netdemand-20220602.csv', './scraped-data/daily/')
df = set_dt_index_old(df, date="06/02/2022")




df.index

289
289
289
288


DatetimeIndex(['2022-06-02 00:00:00', '2022-06-02 00:05:00',
               '2022-06-02 00:10:00', '2022-06-02 00:15:00',
               '2022-06-02 00:20:00', '2022-06-02 00:25:00',
               '2022-06-02 00:30:00', '2022-06-02 00:35:00',
               '2022-06-02 00:40:00', '2022-06-02 00:45:00',
               ...
               '2022-06-02 23:10:00', '2022-06-02 23:15:00',
               '2022-06-02 23:20:00', '2022-06-02 23:25:00',
               '2022-06-02 23:30:00', '2022-06-02 23:35:00',
               '2022-06-02 23:40:00', '2022-06-02 23:45:00',
               '2022-06-02 23:50:00', '2022-06-02 23:55:00'],
              dtype='datetime64[ns]', name='Timestamp', length=288, freq=None)

In [81]:
# df = pd.read_csv('./scraped-data/daily/CAISO-netdemand-20210817.csv').iloc[:, :-1].transpose()
# new_header = df.iloc[0]
# df = df[1:]
# df.columns = new_header
# df.drop("Day-ahead net forecast", axis=1, inplace=True)
# df

def get_df_new(file:str, path:str='')-> pd.DataFrame: 
    if path: 
        file = path + file
    df = pd.read_csv(file).iloc[:, :-1].transpose()
    new_header = df.iloc[0]
    df = df[1:]
    df.columns = new_header
    df.drop("Day-ahead net forecast", axis=1, inplace=True)
    return df

def set_dt_index_new(df:pd.DataFrame, date:str)->pd.DataFrame:
    index = list(map(lambda x: datetime.strptime(date + ' ' + x, "%m/%d/%Y %H:%M"), list(df.index)))
    df.index=pd.DatetimeIndex(index)
    df.index.names = ['Timestamp']
    return df

file = 'CAISO-netdemand-20210817.csv'
path = './scraped-data/daily/'
df = get_df_new(file=file, path=path)
df = set_dt_index(df, '08/17/2021')
df.to_csv('./text.csv')
df.index

FileNotFoundError: [Errno 2] No such file or directory: './scraped-data/daily/CAISO-netdemand-20210817.csv'

In [93]:
df = pd.read_csv('./scraped-data/caiso-net-demand.csv')
df


Timestamp  Hour-ahead forecast   Demand  Net demand
0       2018-04-10 00:00:00              22701.0  22459.0     22117.0
1       2018-04-10 00:05:00              22311.0  22371.0     21955.0
2       2018-04-10 00:10:00              22311.0  22330.0     21882.0
3       2018-04-10 00:15:00              22311.0  22082.0     21774.0
4       2018-04-10 00:20:00              22094.0  21978.0     21613.0
...                     ...                  ...      ...         ...
353371  2021-08-18 23:35:00              26615.0  26982.0     22782.0
353372  2021-08-18 23:40:00              26615.0  26808.0     22634.0
353373  2021-08-18 23:45:00              26615.0  26672.0     22545.0
353374  2021-08-18 23:50:00              26236.0  26506.0     22421.0
353375  2021-08-18 23:55:00              26236.0  26321.0     22220.0

[353376 rows x 4 columns]